In [4]:
import pandas as pd
import numpy as np
import pickle
#import nltk
#import re
#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

ges/pkg_resources/__init__.py", line 3251, in _initialize_master_working_set
    working_set = WorkingSet._build_master()
  File "/opt/anaconda3/envs/social_graphs_project/lib/python3.7/site-packages/pkg_resources/__init__.py", line 558, in _build_master
    ws = cls()
  File "/opt/anaconda3/envs/social_graphs_project/lib/python3.7/site-packages/pkg_resources/__init__.py", line 551, in __init__
    self.add_entry(entry)
  File "/opt/anaconda3/envs/social_graphs_project/lib/python3.7/site-packages/pkg_resources/__init__.py", line 607, in add_entry
    for dist in find_distributions(entry, True):
  File "/opt/anaconda3/envs/social_graphs_project/lib/python3.7/site-packages/pkg_resources/__init__.py", line 1961, in find_distributions
    importer = get_importer(path_item)
  File "/opt/anaconda3/envs/social_graphs_project/lib/python3.7/pkgutil.py", line 419, in get_importer
    importer = path_hook(path_item)
  File "<frozen importlib._bootstrap_external>", line 1447, in path_hook_for_File

In [ ]:
df = pd.read_csv('../Data/Interim/hydrated_Tweet200316.csv', index_col=0)

In [ ]:
# Clean tweets
df['CleanTweet'] = [re.sub("[^a-zA-Z0-9\s,.-_´&%'\":€$£!?']", '',  re.sub(' http\S+', '', re.sub('\s',' ', tw))).replace(u'\xa0', u' ').lower() if isinstance(tw, str) else '' for tw in df.tweet]

# Get bigrams
bigrams_tweets = [list(nltk.bigrams(tw)) for tw in df['CleanTweet']]

In [ ]:
with open('../Data/Processed/Bigram_tweets.pickle', 'wb') as handle:
    pickle.dump(bigrams_tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
unique_chars = sorted(''.join(set(''.join(df['CleanTweet'])))) # Assume all characters are in tweets from df1

unique_bigrams = [x+y for x in unique_chars for y in unique_chars]

bigram_mapper = dict(zip(unique_bigrams, range(len(unique_bigrams))))

In [ ]:
X = vectorizer.fit_transform(df['CleanTweet'])

In [ ]:
X.shape

In [ ]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
lda.fit(X[:10000])

In [ ]:
topics = lda.transform(X[:10000]).argmax(axis = 1)

In [ ]:
y = np.bincount(topics)
ii = np.nonzero(y)[0]
dict(zip(ii,y[ii]))

In [ ]:
lda.transform(X[:10000])

In [ ]:
lda.score(X[:10000])

In [ ]:
vectorizerTfidf = TfidfVectorizer(vocabulary = bigram_mapper, ngram_range = (2,2), analyzer = "char")

In [ ]:
X = vectorizerTfidf.fit_transform(df['CleanTweet'])

In [ ]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
lda.fit(X[:10000])

In [ ]:
lda.score(X[:10000])

# With the data

In [4]:
with open('../Data/Processed/data_til_toke.pkl', 'rb') as handle:
    data = pickle.load(handle)

bigram_mapper = data['bigram_mapper']
list_o_lists = data['list_o_lists']

del data

ERROR! Session/line number was not unique in database. History logging moved to new session 612


In [14]:
lda = LatentDirichletAllocation(n_components=5, verbose = 2, n_jobs = 3)
